##### Imports the necessary modules and sets the system path to locate them.

In [ ]:
import sys
sys.path.append("../utils")
from astra_sim import AstraSim, Collective, NetworkBackend

##### Connects the client to the AstraSim gRPC server, initializes the AstraSim SDK, and creates a folder (tagged as specified) containing all configuration details, generated results, and logs.

In [ ]:
astra = AstraSim(server_endpoint = "172.17.0.2:8989", tag = "analytical_aware_trial")


##### Generates workload execution traces for each rank and configures the data size, which is mandatory for AstraSim workload configuration.

In [ ]:
astra.configuration.common_config.workload = astra.generate_collective(collective=Collective.ALLTOALL, coll_size= 8*1024*1024, npu_range=[0,8])
print(astra.configuration.common_config.workload)


##### Configure the system configurations

In [ ]:
astra.configuration.common_config.system.scheduling_policy = astra.configuration.common_config.system.LIFO
astra.configuration.common_config.system.endpoint_delay = 10
astra.configuration.common_config.system.active_chunks_per_dimension = 1
astra.configuration.common_config.system.all_gather_implementation = [astra.configuration.common_config.system.RING]
astra.configuration.common_config.system.all_to_all_implementation = [astra.configuration.common_config.system.DIRECT]
astra.configuration.common_config.system.all_reduce_implementation = [astra.configuration.common_config.system.RING]
astra.configuration.common_config.system.collective_optimization = astra.configuration.common_config.system.LOCALBWAWARE
astra.configuration.common_config.system.local_mem_bw = 1600
print(astra.configuration.common_config.system)


##### Configure the remote memory configuration

In [ ]:
astra.configuration.common_config.remote_memory.memory_type = astra.configuration.common_config.remote_memory.NO_MEMORY_EXPANSION
print(astra.configuration.common_config.remote_memory)


##### Configure the Network_backend and topology

In [ ]:
# astra.configuration.network_backend.choice = astra.configuration.network_backend.ANALYTICAL_CONGESTION_AWARE
astra.configuration.network_backend.analytical_congestion_aware.topology.network.clear()
astra.configuration.network_backend.analytical_congestion_aware.topology.network.add("switch", 8, 100, 0.005) # add(type_of_topology, number_of_nodes, bandwidth_in_gbps, latency_in_ns)
print("Network backend set to", astra.configuration.network_backend.choice)
print("network backend choice set to:",astra.configuration.network_backend.analytical_congestion_aware.topology.choice)


##### Configure the cmd parameters, non-mandatory parameters

In [ ]:
astra.configuration.common_config.cmd_parameters.comm_scale = 1
astra.configuration.common_config.cmd_parameters.rendezvous_protocol = False
astra.configuration.common_config.cmd_parameters.injection_scale = 1

print(astra.configuration.network_backend.analytical_congestion_aware.topology.network)


#### Start the simulation by providing the network backend name in uppercase letters.

In [ ]:
astra.run_simulation(NetworkBackend.ANALYTICAL_CONGESTION_AWARE)

##### Download all the configurations as a zip

In [ ]:
astra.download_configuration()